# Orbits of exoplanets and free-floating planets

In [1]:
import matplotlib as plt
class Body():
    # n = number of bodies (star and planets)
    G=6.67e-11
    n = 0
    def __init__(self, pos, vel, mass):

        # i = particle index
        self.i = Body.n
        Body.n += 1

        self.m = mass        
        # position, velocity, acceleration  
        self.pos = pos
        self.vel = vel
        self.acc = 0
        
        self.half_pos =0
        self.half_vel =0
        self.half_acc =0
        
    # compute particle acceleration using Newton's law of universal gravitation
    # we need to add the contributions for all other bodies 
    def calculate_acc(self,bodies):
        self.acc=0
        for j in range(len(bodies)):
            if self.i !=j:
                self.acc      += (bodies[j].pos       - self.pos)   *bodies[j].m*G* (1/(((self.pos[0]     -bodies[j].pos[0])**2       + (self.pos[1]-bodies[j].pos[1])**2)**(3/2)))

    def calculate_half_acc(self,bodies):
        self.half_acc=0
        for j in range(len(bodies)):
            if self.i !=j:
                self.half_acc += (bodies[j].half_pos - self.half_pos)*bodies[j].m*G* (1/(((self.half_pos[0]-bodies[j].half_pos[0])**2 + (self.half_pos[1]-bodies[j].half_pos[1])**2)**(3/2)))



In [2]:

import numpy as np
import random 
#bodies list

bodies = []

#set number of planets
nplanets=random.randint(1,5)
nplanets=1
#add star in number of bodies
nbodies=nplanets+1

#set planet mass (in units of Jupiter masses)

#define gravitational constant
G=6.67e-11

#defiine Solar mass (SI)
solar_mass=2e30
#define Jupiter mass (SI)
jupiter_mass=0.10045*solar_mass
#define AU (SI)
AU=1.496e11

def calculate_velocity(distance,mass):
    return(np.sqrt(G*mass/distance))

#set stellar properties

#in Solar masses
stellar_mass=1
#convert to SI
stellar_mass=stellar_mass*solar_mass
position=np.zeros(2)
velocity=np.zeros(2)
    
bodies.append(Body(pos = position, vel = velocity , mass = stellar_mass))

for i in range(1,nbodies):  
    #mass in Jupiter masses
    mass=float(random.randint(1, 10))
    #orbital radius (distance from star) in AU
    orbital_radius=float(random.randint(1, 100))
    orbital_radius=5
    #convert to SI units
    orbital_radius=orbital_radius*AU
    mass=mass*jupiter_mass
    mass=0.001*solar_mass

    position=np.zeros(2)
    velocity=np.zeros(2)

    position[0]=orbital_radius
    velocity[1]=calculate_velocity(orbital_radius,bodies[0].m)
    bodies.append(Body(pos = position, vel = velocity , mass = mass))
     
print(nplanets)
 

1


In [3]:
print(bodies[0].m)
print(bodies[1].m)

print(len(bodies))

print(bodies[1].pos)
print(bodies[1].vel)
print(bodies[0].pos)
print(bodies[0].vel)

2e+30
2e+27
2
[7.48e+11 0.00e+00]
[    0.         13354.48411543]
[0. 0.]
[0. 0.]


In [4]:
time=0
endtime=20
dt=0.1
print_dt=0.1
output_time=0
steps=0
year_to_sec=365*24*60*60

#convert to SI
endtime=endtime*year_to_sec
dt=dt*year_to_sec
print_dt=print_dt*year_to_sec
steps=0
n=len(bodies)


output= open("orbits.txt","w")

while time<endtime:

    for i in range(n):
        bodies[i].calculate_acc(bodies)

    for i in range(n):
        if i==1 and time>=output_time:
            output.write("%10.3e , %10.3e\n" % ((bodies[i].pos[0]-bodies[0].pos[0])/AU, (bodies[i].pos[1]-bodies[0].pos[1])/AU) )
            output_time=output_time+print_dt
            steps+=1
        bodies[i].half_pos=bodies[i].pos+bodies[i].vel*dt/2.
        bodies[i].half_vel=bodies[i].vel+bodies[i].acc*dt/2.

    for i in range(n):
        bodies[i].calculate_half_acc(bodies)
        bodies[i].pos=bodies[i].pos+bodies[i].half_vel*dt
        bodies[i].vel=bodies[i].vel+bodies[i].half_acc*dt
        
    #write time and positions of bodies in a file (for visualization)

    time += dt
    
output.close()
 

In [5]:
# Visualization of the solution with matplotlib. It uses a slider to change the time
################################################################################################################################
import matplotlib.pyplot as plt
from IPython import get_ipython
import pandas as pd
import sys

get_ipython().run_line_magic('matplotlib', 'qt')

data = pd.read_csv('orbits.txt', delimiter=",",usecols=range(2), names=['x','y'], skiprows=[0])

x=np.array(data['x'])
y=np.array(data['y'])


plt.style.use('dark_background')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(1,1,1)

plt.subplots_adjust(bottom=0.2,left=0.15)

ax.axis('equal')
ax.axis([-10, 10, -10, 10])
#ax.set_title('Energy =' + str(bodies.n))
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
print('n=',n)

circle = [None]*n
line  = [None]*n
n=1
for i in range(n):
    print(i)
    circle[i] = plt.Circle((x[0], y[0]), 0.08, ec="w", lw=2.5, zorder=20)
    ax.add_patch(circle[i])
    line[i] = ax.plot(x[0],y[0])[0]
    
from matplotlib.widgets import Slider

slider_ax = plt.axes([0.1, 0.05, 0.8, 0.05])
slider = Slider(slider_ax,      # the axes object containing the slider
                  't',            # the name of the slider parameter
                  0,          # minimal value of the parameter
                  endtime/year_to_sec,          # maximal value of the parameter
                  valinit=0,  # initial value of the parameter 
                  color = '#5c05ff' 
                 )
print(x[0])

print(int(np.rint(10./(dt/year_to_sec))))
def update(timer):
    i = int(np.rint(timer/(dt/year_to_sec)))
    #ax.set_title('Energy =' + str(Energy[i]))
    n=1
    for j in range(n):
        circle[j].center = x[i], y[i]
        line[j].set_xdata(x[:i+1])
        line[j].set_ydata(y[:i+1])
        # I need to have all positions in one array

slider.on_changed(update)

plt.show()

n= 2
0
4.992
100
Traceback (most recent call last):
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/widgets.py", line 440, in _update
    self.set_val(val)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/widgets.py", line 474, in set_val
    func(val)
  File "<ipython-input-5-fa52da8cdb9e>", line 56, in update
    circle[j].center = x[i], y[i]
IndexError: index 200 is out of bounds for axis 0 with size 199
